In [1]:
import os.path
import mushr_rhc.defaults.parameters
import mushr_rhc.defaults.logger
import mushr_rhc.utils
import torch
import yaml
import pickle

push_data_collector = os.path.expanduser("~/catkin_ws/src/pixel_art/push_data_collector")
push_measures = os.path.expanduser("~/catkin_ws/src/pixel_art/push_data_collector/params/pusher_measures.yaml")
with open(push_measures, 'r') as f:
    push_measures_yaml = yaml.load(f)

K = 5
T = 20

params_dict = {
    "K" : K,
    "T": T,
    "horizon/time": .33,
    "npos": 6,
    "trajgen/desired_speed": 0.44,
    "x_gpr_file": "%s/x_gpr_jan_26.pickle" % push_data_collector,
    "y_gpr_file": "%s/y_gpr_jan_26.pickle" % push_data_collector,
    "theta_gpr_file": "%s/t_gpr_jan_26.pickle" % push_data_collector,
    "pushes_start_file": "%s/pushes_x.pickle" % push_data_collector,
    "pushes_file": "%s/pushes_y.pickle" % push_data_collector,
}

params_dict.update(push_measures_yaml)

params = mushr_rhc.defaults.parameters.DictParams(params_dict)

with open(params.get_str("x_gpr_file"), "rb") as f:
    x_gpr = pickle.load(f)

with open(params.get_str("y_gpr_file"), "rb") as f:
    y_gpr = pickle.load(f)

with open(params.get_str("theta_gpr_file"), "rb") as f:
    t_gpr = pickle.load(f)

In [2]:
import mushr_rhc.trajgen
import mushr_rhc.model

logger = mushr_rhc.defaults.logger.StdLog()

gpr = mushr_rhc.model.GPR(params, logger, torch.FloatTensor, gprs=(x_gpr, y_gpr, t_gpr))
traj_gen = mushr_rhc.trajgen.TL(params, logger, torch.FloatTensor)

In [4]:
%matplotlib tk
import numpy as np
import math
import matplotlib.pyplot as plt

colors = ["red", "green", "blue", "black", "orange"]
arrows = True

def plot_block(x, y, theta, color):
    b= float(push_measures_yaml["block_side_len"]) / 2.0
    s, c = math.sin(theta), math.cos(theta)
    b1 = np.array([(c * -b) + (-s * -b) + x, (s * -b) + (c * -b) + y])
    b2 = np.array([(c * -b) + (-s * b) + x, (s * -b) + (c * b) + y])
    b3 = np.array([(c * b) + (-s * b) + x, (s * b) + (c * b) + y])
    b4 = np.array([(c * b) + (-s * -b) + x, (s * b) + (c * -b) + y])
    plt.plot([b1[0], b2[0]], [b1[1], b2[1]], color=color)
    plt.plot([b2[0], b3[0]], [b2[1], b3[1]], color=color)
    plt.plot([b3[0], b4[0]], [b3[1], b4[1]], color=color)
    plt.plot([b4[0], b1[0]], [b4[1], b1[1]], color=color)

def plot_pusher(car_x, car_y, car_theta, color):
    p_x = float(push_measures_yaml["x_pusher_pos"])
    p_y_top = float(push_measures_yaml["y_pusher_top"])
    p_y_bot = float(push_measures_yaml["y_pusher_bot"])
    s, c = math.sin(car_theta), math.cos(car_theta)
    b1 = np.array([(c * p_x) + (-s * p_y_bot) + car_x, (s * p_x) + (c * p_y_bot) + car_y])
    b2 = np.array([(c * p_x) + (-s * p_y_top) + car_x, (s * p_x) + (c * p_y_top) + car_y])
    plt.plot([b1[0], b2[0]], [b1[1], b2[1]], color=color)


#for xd in np.arange(0.29, .6, 0.08):
for xd in [0.29]:
    print "---- XD: ", xd
    t = traj_gen.get_control_trajectories(0.5)
    # pose = torch.FloatTensor([0.0, 0.0, np.pi/2, 0.1, xd, math.pi/4])
    pose = torch.FloatTensor([0.0, 0.0, 0.0, xd, 0.1, math.pi/4])

    rollouts = torch.FloatTensor(K, T, 6)

    gpr.rollout(pose, t, rollouts)

    i = 0
    for r in rollouts:
        plt.figure()
        plt.scatter(r[:, 0], r[:, 1], color=colors[i])
        plt.plot(r[:, 0], r[:, 1], color=colors[i])
        for p in r:
            plot_pusher(p[0], p[1], p[2], color=colors[i])

        #plt.scatter(rollouts[:, 0, 3], rollouts[:, 0, 4], color="red")

        if arrows:
            al = 0.005
            for p in r:
                # print p[3], p[4], al * torch.cos(p[5]), al * torch.sin(p[5])
                #plt.arrow(p[3], p[4], al * torch.cos(p[5]), al * torch.sin(p[5]), color="black", width=0.00005)
                plot_block(p[3], p[4], p[5], colors[i])

        plt.scatter(r[:, 3], r[:, 4], color=colors[i])
        plt.plot(r[:, 3], r[:, 4], color=colors[i])
        i+=1
    plt.show()

---- XD:  0.29
rotated tensor([[0.2900, 0.1000, 0.7854],
        [0.2900, 0.1000, 0.7854],
        [0.2900, 0.1000, 0.7854],
        [0.2900, 0.1000, 0.7854],
        [0.2900, 0.1000, 0.7854]])
diffs tensor([[0.2900, 0.1000, 0.7854],
        [0.2900, 0.1000, 0.7854],
        [0.2900, 0.1000, 0.7854],
        [0.2900, 0.1000, 0.7854],
        [0.2900, 0.1000, 0.7854]])
rotated tensor([[0.2845, 0.1025, 0.7911],
        [0.2848, 0.1012, 0.7882],
        [0.2851, 0.1000, 0.7854],
        [0.2853, 0.0988, 0.7826],
        [0.2857, 0.0975, 0.7797]])
diffs tensor([[0.2851, 0.1009, 0.7911],
        [0.2851, 0.1004, 0.7882],
        [0.2850, 0.1000, 0.7854],
        [0.2851, 0.0996, 0.7826],
        [0.2851, 0.0991, 0.7797]])
rotated tensor([[0.2791, 0.1050, 0.7969],
        [0.2796, 0.1025, 0.7910],
        [0.2801, 0.1000, 0.7854],
        [0.2807, 0.0975, 0.7798],
        [0.2814, 0.0950, 0.7739]])
diffs tensor([[0.2803, 0.1018, 0.7969],
        [0.2801, 0.1009, 0.7910],
        [0.2801, 0.1